In [12]:
from IPython.display import IFrame
IFrame("homework1.pdf", width=600, height=300)

In [2]:
#importing libraries
import numpy as np
import json

In [7]:
#utility function for parsing JSONdata
def parseData(fname):
    for l in fname:
        yield eval(l)

In [6]:
file = open('beer_50000.json')

In [8]:
# loading and parsing JSONdata
data  =list(parseData(file))

In [9]:
type(data)

list


# Tasks — Regression (week 1):

1.No of Reviews for each style of beer in the dataset (`beer/style`)

In [25]:
def countFreqOfBeerStyles(list_of_dict):
    freq = dict()
    for dict_ in list_of_dict:
        if dict_['beer/style'] not in freq:
            freq[dict_['beer/style']] = 0
        freq[dict_['beer/style']] += 1
    return freq

In [26]:
freq_styles = countFreqOfBeerStyles(data)

In [29]:
freq_styles 

{'Altbier': 165,
 'American Adjunct Lager': 242,
 'American Amber / Red Ale': 665,
 'American Amber / Red Lager': 42,
 'American Barleywine': 825,
 'American Black Ale': 138,
 'American Blonde Ale': 357,
 'American Brown Ale': 314,
 'American Dark Wheat Ale': 14,
 'American Double / Imperial IPA': 3886,
 'American Double / Imperial Pilsner': 14,
 'American Double / Imperial Stout': 5964,
 'American IPA': 4113,
 'American Malt Liquor': 90,
 'American Pale Ale (APA)': 2288,
 'American Pale Lager': 123,
 'American Pale Wheat Ale': 154,
 'American Porter': 2230,
 'American Stout': 591,
 'American Strong Ale': 166,
 'American Wild Ale': 98,
 'Baltic Porter': 514,
 'Belgian Dark Ale': 175,
 'Belgian IPA': 128,
 'Belgian Pale Ale': 144,
 'Belgian Strong Dark Ale': 146,
 'Belgian Strong Pale Ale': 632,
 'Berliner Weissbier': 10,
 'BiÃ¨re de Garde': 7,
 'Black & Tan': 122,
 'Bock': 148,
 'Braggot': 26,
 'California Common / Steam Beer': 11,
 'Chile Beer': 11,
 'Cream Ale': 69,
 'Czech Pilsener'

avg value of `review/taste` for reviews from each style  

In [40]:
def avgReview(dict):
    total = dict()
    freq = dict()
    avg = dict()
    for dict_ in data:
        if dict_['beer/style'] not in total:
            total[dict_['beer/style']] = 0
            freq[dict_['beer/style']]  = 0
            
        total[dict_['beer/style']] += dict_['review/taste']
        freq [dict_['beer/style']] += 1
    for style in total:
        avg[style] = round(total[style]/freq[style], 6)
    return avg, freq

In [41]:
avg ,freq = avgReview(dict)

In [42]:
avg

{'Altbier': 3.40303,
 'American Adjunct Lager': 2.948347,
 'American Amber / Red Ale': 3.513534,
 'American Amber / Red Lager': 3.690476,
 'American Barleywine': 4.064242,
 'American Black Ale': 3.873188,
 'American Blonde Ale': 3.254902,
 'American Brown Ale': 3.743631,
 'American Dark Wheat Ale': 3.678571,
 'American Double / Imperial IPA': 4.033325,
 'American Double / Imperial Pilsner': 3.821429,
 'American Double / Imperial Stout': 4.479963,
 'American IPA': 4.000851,
 'American Malt Liquor': 2.255556,
 'American Pale Ale (APA)': 3.649694,
 'American Pale Lager': 3.215447,
 'American Pale Wheat Ale': 3.334416,
 'American Porter': 4.081839,
 'American Stout': 3.819797,
 'American Strong Ale': 3.569277,
 'American Wild Ale': 4.188776,
 'Baltic Porter': 4.213035,
 'Belgian Dark Ale': 3.34,
 'Belgian IPA': 3.949219,
 'Belgian Pale Ale': 3.739583,
 'Belgian Strong Dark Ale': 3.695205,
 'Belgian Strong Pale Ale': 4.056171,
 'Berliner Weissbier': 3.55,
 'BiÃ¨re de Garde': 3.928571,
 'Bla

In [39]:
freq

{'Altbier': 165,
 'American Adjunct Lager': 242,
 'American Amber / Red Ale': 665,
 'American Amber / Red Lager': 42,
 'American Barleywine': 825,
 'American Black Ale': 138,
 'American Blonde Ale': 357,
 'American Brown Ale': 314,
 'American Dark Wheat Ale': 14,
 'American Double / Imperial IPA': 3886,
 'American Double / Imperial Pilsner': 14,
 'American Double / Imperial Stout': 5964,
 'American IPA': 4113,
 'American Malt Liquor': 90,
 'American Pale Ale (APA)': 2288,
 'American Pale Lager': 123,
 'American Pale Wheat Ale': 154,
 'American Porter': 2230,
 'American Stout': 591,
 'American Strong Ale': 166,
 'American Wild Ale': 98,
 'Baltic Porter': 514,
 'Belgian Dark Ale': 175,
 'Belgian IPA': 128,
 'Belgian Pale Ale': 144,
 'Belgian Strong Dark Ale': 146,
 'Belgian Strong Pale Ale': 632,
 'Berliner Weissbier': 10,
 'BiÃ¨re de Garde': 7,
 'Black & Tan': 122,
 'Bock': 148,
 'Braggot': 26,
 'California Common / Steam Beer': 11,
 'Chile Beer': 11,
 'Cream Ale': 69,
 'Czech Pilsener'